<a href="https://colab.research.google.com/github/kbrh3/AI-algorithm-practice/blob/main/461SearchMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Questions
1. Given a list of cities and their adjacencies—from city A, what cities are next to it—can a route be found
from city A to city X?
2. utilize your resources to rapidly implement a version of each of the major methods we've discussed (some are pending) in chapter 2.
3. create a report detailing your findings of a comparison of each of the following methods:


*   Breadth-Search Force
*   depth-search force
*   ID-DFS search
*   best-first search
*   A* star search







In [ ]:
import time
import csv
import math
import heapq
import sys
import matplotlib.pyplot as plt
!pip install adjustText
import adjustText
!pip install pympler
from pympler import asizeof
from collections import deque


In [ ]:

#read in scv coordinates file
def read_town_co(co):
    town_co = {} #dictionary of towns and coordinates

    with open(co, 'r') as file: #read in file to dictionary
        csv_reader = csv.reader(file)
        for row in csv_reader:
            city = row[0]
            latitude = float(row[1])
            longitude = float(row[2])
            town_co[city] = (latitude, longitude)

    return town_co

co = '/content/coordinates (1).csv'
town_co = read_town_co(co)

# Print to check - works 10/4/24
#for city, coords in town_co.items():
#    print(f"{city}: Latitude {coords[0]}, Longitude {coords[1]}")


In [ ]:
# Function reads adjacency file & create adjacency list
def adj_file(adj):
    adj_list = {}

    with open(adj, 'r') as file:
        for line in file:
            # Split the line by spaces - to extract the city pairs * important 10/4/24
            city_a, city_b = line.strip().split()

            # Add connection in BOTH!!! directions
            if city_a not in adj_list:
                adj_list[city_a] = []
            if city_b not in adj_list:
                adj_list[city_b] = []

            adj_list[city_a].append(city_b)
            adj_list[city_b].append(city_a)

    return adj_list
#test
adj = '/content/Adjacencies.txt'
adj_list = adj_file(adj)

# Print to test - works 10/5/24
#for city, neighbors in adj_list.items():
 #   print(f"{city}: {', '.join(neighbors)}")


###functions to assist in search methods + UI

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

#plot the route using networkx
def networkx(path, town_co, adj_list):
    G = nx.Graph()

    #add nodes with coordinates as positions
    pos = {town: (lon, lat) for town, (lat, lon) in town_co.items()}
    G.add_nodes_from(pos.keys())

    #Add all edges from adjacency list
    for city, neighbors in adj_list.items():
        for neighbor in neighbors:
            G.add_edge(city, neighbor)

     # Use spring_layout to add space between nodes "GPT assistance 10/6/24, assisted in formatting"
    pos = nx.spring_layout(G, k=0.25, iterations=50)  # Adjust 'k' for more space between nodes
    plt.figure(figsize=(12, 10))  # Larger figure size for better visibility

    #Plot nodes w/ their labels
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='#add8e6', font_size=10, font_color='black', edge_color='gray', style='dashed')

    #Highlight route found
    if path:
        route_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
        nx.draw_networkx_edges(G, pos, edgelist=route_edges, edge_color='red', width=3.5)

    plt.title('Route Visualization (NetworkX)')
    plt.show()




In [ ]:
#Function to plot cities and the route on a 2D map with **automatic** label adjustment
#still having trouble with formatting, maybe try the non-automatic version?? 10/9/24
from adjustText import adjust_text
#scale factor is what creates more space between nodes
def plot_2droute(path, town_co, adj_list, scale_factor):
    fig, ax = plt.subplots(figsize=(12, 10))  #Increased figure size- better readability

    #artificial spacing factor 2 spread out the cities
    def adjust_position(value, factor):
        return value * (1 + factor)

    texts = []  #List to store text objects for adjustment

    #Plot all cities as points w/ adjusted spacing
    for town, (lat, lon) in town_co.items():
        #Adjust longitude and latitude for better spacing
        scaled_lon = adjust_position(lon, scale_factor)
        scaled_lat = adjust_position(lat, scale_factor)
        ax.scatter(scaled_lon, scaled_lat, c='#add8e6', s=30, edgecolors='black', zorder=3)  # Lighter blue color, smaller size

        #Create text labels and store in list
        texts.append(ax.text(scaled_lon, scaled_lat, town, fontsize=9, ha='right', color='black'))

    #Plot all edges as lines w/ adjusted spacing
    #claude AI assisted in formatting here 10/8/24
    for city, neighbors in adj_list.items():
        for neighbor in neighbors:
            city_lat, city_lon = town_co[city]
            neighbor_lat, neighbor_lon = town_co[neighbor]
            #Apply scaling to city and neighbor coordinates
            scaled_city_lon = adjust_position(city_lon, scale_factor)
            scaled_city_lat = adjust_position(city_lat, scale_factor)
            scaled_neighbor_lon = adjust_position(neighbor_lon, scale_factor)
            scaled_neighbor_lat = adjust_position(neighbor_lat, scale_factor)
            ax.plot([scaled_city_lon, scaled_neighbor_lon], [scaled_city_lat, scaled_neighbor_lat], 'gray', linestyle='--', linewidth=1)

    #Highlight the path
    if path:
        for i in range(len(path) - 1):
            city_lat, city_lon = town_co[path[i]]
            next_city_lat, next_city_lon = town_co[path[i + 1]]
            #Apply scaling to path coordinates - Claude AI assisted 10/9/24
            scaled_city_lon = adjust_position(city_lon, scale_factor)
            scaled_city_lat = adjust_position(city_lat, scale_factor)
            scaled_next_city_lon = adjust_position(next_city_lon, scale_factor)
            scaled_next_city_lat = adjust_position(next_city_lat, scale_factor)
            ax.plot([scaled_city_lon, scaled_next_city_lon], [scaled_city_lat, scaled_next_city_lat], 'red', linewidth=2.5)  # Highlight the path

    #"Adjust the positions of all text objects to avoid overlap" - sort of works??
    adjust_text(texts)

    #equal scaling & grid for clarity
    plt.axis('equal')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.set_title('Route on 2D Map (Latitude vs Longitude)', fontsize=14)
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)

    plt.show()


In [ ]:
#from pympler import asizeof - fixed asizeoferror 10/10/24
from pympler.asizeof import asizeof
# Test memory usage of a simple object
test_list = [1, 2, 3, 4, 5]
print(f"Memory used: {asizeof(test_list) / 1024:.2f} KB")  # Check if asizeof works here


Memory used: 0.26 KB


In [ ]:
#memory management - determine the total memory used (scale of the arrays) to find the solution.
#optional, do this last
from pympler.asizeof import asizeof  #asizeof is correctly imported!!!

def memory(path, adj_list, town_co):
    total_mem = asizeof(adj_list) + asizeof(town_co) + asizeof(path)
    print(f"Total memory used: {total_mem / 1024:.2f} KB")  # Convert bytes to KB



In [ ]:
#prints towns out in menu/UI
def print_towns(town_co):
    towns = list(town_co.keys())
    num_towns = len(towns)

    #Calculate the num of rows needed to evenly distribute towns across 3 columns
    num_columns = 3
    rows = (num_towns + num_columns - 1) // num_columns  #Rounds up number of rows

    # Print towns in 3 columns - GPT created this portion
    for row in range(rows):
        # Print each row across the 3 columns
        col1 = towns[row] if row < len(towns) else ""
        col2 = towns[row + rows] if row + rows < len(towns) else ""
        col3 = towns[row + 2 * rows] if row + 2 * rows < len(towns) else ""
        print(f"{col1:<20} {col2:<20} {col3:<20}")  #Use formatting for even spacing
        print("-" * 60)  #Print a separator line - makes it easier to read


In [ ]:
#distance calculating function - GPT assisted in formatting and logic
def calc_distance(path, town_co, max_time=5): #max time is the timeout function
    start_time = time.time() #not using this for display right now, kept just in case (below)
    total_distance = 0

    for i in range(len(path) - 1):
        current_time = time.time()
        elapsed_time = current_time - start_time #watch time

        if elapsed_time > max_time:  #Check for time-out
            print(f"Distance calculation timed out after {max_time} seconds.")
            return None  #Time-out happens

        city_a = path[i]
        city_b = path[i + 1]
        lat1, lon1 = town_co[city_a]
        lat2, lon2 = town_co[city_b]

        distance = haversine(lat1, lon1, lat2, lon2)
        total_distance += distance
        print(f"Distance from {city_a} to {city_b}: {distance:.2f} km")

    total_time = time.time() - start_time
    print(f"Total distance: {total_distance:.2f} km") #print function here so we don't have to worry about it
    #print(f"Time taken in distance calculation: {total_time:.4f} seconds") #- not using rn
    return total_distance #return this value in case we need it later

In [ ]:
# Function to calculate Haversine distance between two cities
# GPT created - 10/8/24
#prompt "help me build a heversine function that I can use to check distance between cities for my search functions"
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Distance in kilometers

##methods of search

In [ ]:
#brute-force - just realized this one may not be needed, but we will keep it anyway. whoops 10/9/24
def brute_force(start, end, adj_list):
    #Helper function -  recursive search?? gpt assistance here
    def search(path):
        current_city = path[-1]


        #If at goal, return path and print the final path
        if current_city == end:
            return path

        #check all adjacent cities
        for neighbor in adj_list.get(current_city, []):
            if neighbor not in path:  # Avoid cycles (revisiting cities)
                new_path = search(path + [neighbor])
                if new_path:  # If a valid path is found, return it
                    return new_path

        #If none found
        return None

    #Start search with starting city
    return search([start])

#test cities- hardcoded
start_city = "South_Haven"
end_city = "Oxford"

#time checkers - before and after calling function

start_time = time.time()
path = brute_force(start_city, end_city, adj_list)
end_time = time.time()

if path:
    print(f"Path found: {' -> '.join(path)}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)
else:
    print(f"No path found from {start_city} to {end_city}")

print(f"Time taken: {end_time - start_time:.4f} seconds")


Path found: South_Haven -> Caldwell -> Wellington -> Mayfield -> Oxford
Distance from South_Haven to Caldwell: 18.96 km
Distance from Caldwell to Wellington: 27.49 km
Distance from Wellington to Mayfield: 2.85 km
Distance from Mayfield to Oxford: 33.05 km
Total distance: 82.34 km
Total memory used: 23.76 KB
Time taken: 0.0487 seconds


In [ ]:
from collections import deque
# Depth-Limited Search (DLS) used within ID-DFS
#gpt generated, altered to fit style of the rest of my functions
#prompt: "Provide me the basic code of a ID-DFS search algorithm and add comments explaining each step"
def dls(start, end, adj_list, limit):
    # Initialize stack w/ starting city and depth
    stack = deque([(start, [start], 0)])  # (current_city, path, current_depth)

    while stack:
        current_city, path, depth = stack.pop()

        # If we reach the goal, return the path
        if current_city == end:
            return path

        # If depth limit is not reached, explore further
        if depth < limit:
            for neighbor in adj_list.get(current_city, []):
                if neighbor not in path:  # Avoid cycles
                    new_path = path + [neighbor]  # Create new path with neighbor
                    stack.append((neighbor, new_path, depth + 1))  # Push new path onto stack with incremented depth

    # Return None if the goal was not found within the depth limit
    return None

# Iterative Deepening Depth-First Search (ID-DFS)
def iddfs(start, end, adj_list, max_depth=100):
    for limit in range(1, max_depth + 1):
        path = dls(start, end, adj_list, limit)
        if path:  # If a valid path is found, return it
            return path
    # If no path found within max depth
    return None

# Test cities - hardcoded, change to user input
start = "South_Haven"
end = "Oxford"

start_time = time.time()
path = iddfs(start, end, adj_list, max_depth=10)  # Set a reasonable max depth
end_time = time.time()

if path:
    print(f"Path found: {' -> '.join(path)}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)
else:
    print(f"No path found from {start} to {end}")

print(f"Time taken: {end_time - start_time:.4f} seconds")


Path found: South_Haven -> Mulvane -> Mayfield -> Oxford
Distance from South_Haven to Mulvane: 50.33 km
Distance from Mulvane to Mayfield: 36.02 km
Distance from Mayfield to Oxford: 33.05 km
Total distance: 119.39 km
Total memory used: 23.68 KB
Time taken: 0.0002 seconds


In [ ]:
#gpt generated, altered to fit style of the rest of my functions
#prompt: "Provide me the basic code of a breadth first search and add comments explaining each step"
#Breadth-First Search
def bfs(start, end, adj_list):
    #initialize queue w/ starting city
    queue = deque([[start]])
    visited = set([start])  #track visited cities

    while queue:
        # Dequeue the front path
        path = queue.popleft()
        current_city = path[-1]

        # If we reach the goal, return the path
        if current_city == end:
          #print(f"Path found: {' -> '.join(path)}")
          return path

        # Explore all adjacent cities
        for neighbor in adj_list.get(current_city, []):
            if neighbor not in visited:
                visited.add(neighbor)  # Mark city as visited
                new_path = path + [neighbor]  # Create new path with neighbor
                queue.append(new_path)  # "Enqueue the new path"

    # If goal not found
    return None

#test cities - hardcoded to test
start = "South_Haven"
end = "Oxford"

start_time = time.time()
path = bfs(start,end,adj_list)
end_time = time.time()

if path:
    print(f"Path found: {' -> '.join(path)}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)

else:
    print(f"No path found from {start} to {goal}")

print(f"Time taken: {end_time - start_time:.4f} seconds")


Path found: South_Haven -> Caldwell -> Wellington -> Oxford
Distance from South_Haven to Caldwell: 18.96 km
Distance from Caldwell to Wellington: 27.49 km
Distance from Wellington to Oxford: 30.29 km
Total distance: 76.73 km
Total memory used: 23.69 KB
Time taken: 0.0001 seconds


In [ ]:
# Depth-First Search
#gpt generated, altered to fit style of the rest of my functions
#prompt: "Provide me the basic code of a depth first search and add comments explaining each step"
def dfs(start, end, adj_list):
    #Initialize stack w/ starting city
    stack = deque([[start]])
    visited = set([start])  # Track visited cities

    while stack:
        # Pop the top path
        path = stack.pop()
        current_city = path[-1]

        # If we reach the goal, return the path
        if current_city == end:
            return path

        # Explore all adjacent cities
        for neighbor in adj_list.get(current_city, []):
            if neighbor not in visited:
                visited.add(neighbor)  # Mark city as visited
                new_path = path + [neighbor]  # Create new path with neighbor
                stack.append(new_path)  # "Push the new path onto the stack"

    # If goal not found
    return None

# Test cities - hardcoded to test
start = "South_Haven"
end = "Oxford"

start_time = time.time()
path = dfs(start, end, adj_list)
end_time = time.time()

if path:
    print(f"DFS Path: {path}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)
else:
    print(f"No path found from {start} to {end}")

print(f"Time taken: {end_time - start_time:.4f} seconds")

DFS Path: ['South_Haven', 'Mulvane', 'Mayfield', 'Oxford']
Distance from South_Haven to Mulvane: 50.33 km
Distance from Mulvane to Mayfield: 36.02 km
Distance from Mayfield to Oxford: 33.05 km
Total distance: 119.39 km
Total memory used: 23.68 KB
Time taken: 0.0002 seconds


###Heuristic Approaches

In [ ]:
#Best-First Search function
#gpt generated, altered to fit style of the rest of my functions
#prompt: "Provide me the basic code of a breadth first search and add comments explaining each step"
def bestfs(start, end, adj_list, town_co):
    # Initialize priority queue with the starting city and its heuristic value - GPT assisted
    queue = []
    heapq.heappush(queue, (0, [start]))  # (heuristic, path)
    visited = set([start])  # Track visited cities

    while queue:
        # Dequeue the city with the smallest heuristic value
        _, path = heapq.heappop(queue)
        current_city = path[-1]

        # If we reach the goal, return the path
        if current_city == end:
            return path

        # Explore all adjacent cities
        for neighbor in adj_list.get(current_city, []):
            if neighbor not in visited:
                visited.add(neighbor)  # Mark city as visited
                # Calculate heuristic (straight-line distance to the goal)
                h_value = haversine(*town_co[neighbor], *town_co[end])
                new_path = path + [neighbor]  # Create new path with neighbor
                heapq.heappush(queue, (h_value, new_path))  # Push with priority h_value

    # If goal not found
    return None


start = "South_Haven"
end = "Oxford"

start_time = time.time()
path = bestfs(start, end, adj_list, town_co)
end_time = time.time()

if path:
    print(f"Path found: {' -> '.join(path)}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)
else:
    print(f"No path found from {start} to {end}")

print(f"Time taken: {end_time - start_time:.4f} seconds")

Path found: South_Haven -> Mulvane -> Mayfield -> Oxford
Distance from South_Haven to Mulvane: 50.33 km
Distance from Mulvane to Mayfield: 36.02 km
Distance from Mayfield to Oxford: 33.05 km
Total distance: 119.39 km
Total memory used: 23.68 KB
Time taken: 0.0003 seconds


In [ ]:
# A* Search function
#gpt generated, altered to fit style of the rest of my functions
#prompt: "Provide me the basic code of an  A* Search function and add comments explaining each step"
def astar(start, end, adj_list, town_co):
    # Initialize priority queue with the starting city
    queue = []
    heapq.heappush(queue, (0, 0, [start]))  # (total_cost, g_cost, path)
    visited = set([start])  # Track visited cities
    g_costs = {start: 0}  # Track actual cost to reach each city (g(n))

    while queue:
        # Dequeue the city with the smallest total cost
        _, current_g_cost, path = heapq.heappop(queue)
        current_city = path[-1]

        # If we reach the goal, return the path
        if current_city == end:
            return path

        # Explore all adjacent cities
        for neighbor in adj_list.get(current_city, []):
            # Calculate g_cost: the distance from the start to this neighbor
            neighbor_g_cost = current_g_cost + haversine(*town_co[current_city], *town_co[neighbor])

            # If this path to the neighbor is better than any previously recorded path
            if neighbor not in g_costs or neighbor_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = neighbor_g_cost
                # Calculate h_value (heuristic estimate from neighbor to goal)
                h_value = haversine(*town_co[neighbor], *town_co[end])
                # Total cost = g_cost + h_value
                total_cost = neighbor_g_cost + h_value
                new_path = path + [neighbor]  # Create new path with neighbor
                heapq.heappush(queue, (total_cost, neighbor_g_cost, new_path))  # Push with total_cost priority

    # If goal not found
    return None

#test cities - hardcoded
start = "South_Haven"
end = "Oxford"

start_time = time.time()
path = astar(start, end, adj_list, town_co)
end_time = time.time()

if path:
    print(f"Path found: {' -> '.join(path)}")
    distance = calc_distance(path, town_co, max_time=5)
    memory(path, adj_list, town_co)
else:
    print(f"No path found from {start} to {end}")

print(f"Time taken: {end_time - start_time:.4f} seconds")

Path found: South_Haven -> Caldwell -> Wellington -> Oxford
Distance from South_Haven to Caldwell: 18.96 km
Distance from Caldwell to Wellington: 27.49 km
Distance from Wellington to Oxford: 30.29 km
Total distance: 76.73 km
Total memory used: 23.69 KB
Time taken: 0.0003 seconds


###UI


In [ ]:
#List available search methods - includes exit loop option
search_methods = {
    '1': 'Brute-Force',
    '2': 'Breadth-first search',
    '3': 'Depth-first search',
    '4': 'ID-DFS',
    '5': 'Best-First Search',
    '6': 'A* Search',
    '7': 'Exit'
}

# Function to normalize town names for comparison (convert to lowercase and replace spaces with underscores)
def clean_name(town):
    return town.strip().lower().replace(" ", "_")

# Get valid input function with normalization
def get_town_input(town_co, prompt):
    town = input(prompt).strip()

    # Normalize both user input and town names for comparison
    towns = {clean_name(t): t for t in town_co}  # Dictionary with normalized names

    # Check input after normalizing
    while clean_name(town) not in towns:
        print(f"'{town}' is not a valid town. Please select from the list.")
        town = input(prompt).strip()

    # Return the actual town name from the original dataset
    return towns[clean_name(town)]
#get search method from user here
def get_search_method():
    print("\nSelect a search method:")
    for key, method in search_methods.items():
        print(f"{key}. {method}")

    method_choice = input("Enter the number of the search method: ").strip()

    #check input
    while method_choice not in search_methods:
        print(f"{method_choice} is not a valid choice. Please enter a valid number.")
        method_choice = input("Enter the number of the search method: ").strip()

    return search_methods[method_choice]

#Main UI function to get start& end towns from user
#added method comparison 10/7/24
def user_interface(town_co, adj_list):
    #starting and ending towns
    start = get_town_input(town_co, "Enter the starting town: ")
    end = get_town_input(town_co, "Enter the ending town: ")

    print(f"\nYou selected:")
    print(f"Starting town: {start}")
    print(f"Ending town: {end}")

    while True:  #added loop to allow method selection multiple times 10/7/24
        #Get search method from user
        search_method = get_search_method()

        if search_method == 'Exit':
            print("Exiting the search method selection.")
            break  #Exit if the user says to stop

        print(f"\nSearch method: {search_method}")

        #run selected search method and get start time here
        start_time = time.time()

        if search_method == 'Brute-Force':
            path = brute_force(start, end, adj_list)
        elif search_method == 'Breadth-first search':
            path = bfs(start, end, adj_list)
        elif search_method == 'Depth-first search':
            path = dfs(start, end, adj_list)
        elif search_method == 'ID-DFS':
            path = iddfs(start, end, adj_list, max_depth=10)  # Reasonable max depth
        elif search_method == 'Best-First Search':
            path = bestfs(start, end, adj_list, town_co)
        elif search_method == 'A* Search':
            path = astar(start, end, adj_list, town_co)
        else:
            print(f"Invalid method selected. Please try again.")
            continue

        end_time = time.time()

        #Check if path was found + display it
        if path:
            print(f"\n{search_method} Path: {' -> '.join(path)}")
            distance = calc_distance(path, town_co, max_time=5)
            memory(path, adj_list, town_co)
            #plot_2droute(path, town_co, adj_list, scale_factor = 1)
            #these two are taking too long :/
            #networkx(path, town_co, adj_list)

        else:
            print(f"No path found from {start} to {end}")

        print(f"Time taken: {end_time - start_time:.4f} seconds")

        #Ask if user wants to select a new method to compare to previous
        print("\nReturning to search method selection...\n")



In [ ]:
#graph tester:

#test cities and method - copied from above - graph is holding up the loop and I am unsure of how to fix that
method = 'A* Search'
start = "El_Dorado"
end = "Oxford"

if method == 'Brute-Force':
            path = brute_force(start, end, adj_list)
elif method == 'Breadth-first search':
            path = bfs(start, end, adj_list)
elif method == 'Depth-first search':
            path = dfs(start, end, adj_list)
elif method == 'ID-DFS':
            path = iddfs(start, end, adj_list, max_depth=10)  # Reasonable max depth
elif method == 'Best-First Search':
            path = bestfs(start, end, adj_list, town_co)
elif method == 'A* Search':
            path = astar(start, end, adj_list, town_co)

networkx(path, town_co, adj_list) #strange shaking happening here? unsure why, could this be the automatic adjustment for some reason??
#plot_2droute(path, town_co, adj_list, scale_factor=1)

In [ ]:
print_towns(town_co)
user_interface(town_co, adj_list) #call UI

Abilene              Harper               Newton              
------------------------------------------------------------
Andover              Haven                Oxford              
------------------------------------------------------------
Anthony              Hays                 Pratt               
------------------------------------------------------------
Argonia              Hillsboro            Rago                
------------------------------------------------------------
Attica               Hutchinson           Salina              
------------------------------------------------------------
Augusta              Junction_City        Sawyer              
------------------------------------------------------------
Bluff_City           Kingman              South_Haven         
------------------------------------------------------------
Caldwell             Kiowa                Topeka              
------------------------------------------------------------
Cheney  